In [1]:
#!nvidia-smi

In [2]:
import sys
sys.path.append('keras_elmo_bert/')

import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [39]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import itertools
import string
import numpy as np 
import pandas as pd 
import tensorflow as tf
import tensorflow_hub as hub
import re

from keras import backend as K
import keras.layers as layers
from keras.models import Model, load_model
from keras.layers import Layer
from keras.utils import Sequence
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, Flatten, GlobalMaxPool1D, LSTM, Dot, Lambda
from keras.layers import Input, MaxPooling1D, GlobalAveragePooling1D, multiply, concatenate, Reshape
from keras.layers import Bidirectional
from keras.optimizers import Adam
import keras.initializers as initializers
import keras.regularizers as regularizers
import keras.constraints as constraints


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import shuffle
import pickle

sess = tf.Session()

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

In [5]:
max_seq_length = 64
bert_path = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"
elmo_path = "https://tfhub.dev/google/elmo/2"

# Create train test split

Чтобы понять обобщающую способность модели, необходимо валидироватсья не тех обьектах, которые не участвовали в обучении. Соотвественно, мы выбираем некоторое количество индексов и смотрим, чтобы они не находились в валидации в любом из полей qid1 или qid2  

In [6]:
all_ques_df = pd.read_pickle('all_ques_df.pickle')
df = pd.read_pickle('df.pickle')

In [7]:
qid_arr = np.unique(all_ques_df['new_qid'].values)

In [8]:
train_ind, val_ind =  train_test_split(qid_arr, test_size=0.3, random_state=42)

In [9]:
train_df = df[((df.qid1_new.isin(train_ind)) | (df.qid2_new.isin(train_ind)))]

In [10]:
test_df = df[((~df.qid1_new.isin(train_ind)) & (~df.qid2_new.isin(train_ind)))]

In [11]:
print(f'Train size is {len(train_df)/len(df)}')
print(f'Test size is {len(test_df)/len(df)}')

Train size is 0.9112964518580743
Test size is 0.08870354814192567


# Bert pipeline

In [12]:
train_text = train_df[['question1', 'question2']].values
train_label = train_df['is_duplicate'].values

test_text = test_df[['question1', 'question2']].values
test_label = test_df['is_duplicate'].values

## Init model

In [15]:
from bert_tokenizer import BERT_tokenizer

In [16]:
tokenizer = BERT_tokenizer(bert_path,max_seq_length)

In [17]:
(train_input_ids, train_input_masks, train_segment_ids, train_labels 
) = tokenizer.predict(train_text,train_label)
(test_input_ids, test_input_masks, test_segment_ids, test_labels
) = tokenizer.predict(test_text,test_label)


In [19]:
(train_input_ids != 0).sum(axis=1)

array([32, 46, 32, ..., 36, 28, 35])

## Bert model

In [20]:
from bert_layer import BertLayer

In [21]:
def build_model_bert(max_seq_length): 
    in_id = Input(shape=(max_seq_length,), name="input_ids")
    in_mask = Input(shape=(max_seq_length,), name="input_masks")
    in_segment = Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=18, tf_hub = bert_path, output_representation = 'pooled_output', trainable = True)(bert_inputs)
    
    dense = Dense(256, activation='relu')(bert_output)
    pred = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [22]:
bert_model = build_model_bert(max_seq_length)
initialize_vars(sess)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 64)           0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 64)           0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 64)           0                                            
__________________________________________________________________________________________________
bert_layer_1 (BertLayer)        (None, 768)          108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [23]:
bert_model.trainable_weights

[<tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/self/query/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/self/query/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/self/key/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/self/key/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/self/value/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/self/value/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/output/dense/kernel:0' shape=(768, 768) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11/attention/output/dense/bias:0' shape=(768,) dtype=float32>,
 <tf.Variable 'bert_layer_1_module/bert/encoder/layer_11

In [30]:
bert_model.fit(
    [train_input_ids, train_input_masks, train_segment_ids], 
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=5,
    batch_size=256
)

In [29]:
bert_model.save('BertModel.h5')

## Load BERT model

In [31]:
bert_model = build_model_bert(max_seq_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 64)           0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 64)           0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 64)           0                                            
__________________________________________________________________________________________________
bert_layer_3 (BertLayer)        (None, 768)          108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [32]:
bert_model.load_weights('BertModel.h5')

## Find 100 most similar objects for qid =1

In [34]:
all_ques_df.head()

,qid,questions,num_of_words,new_qid
0,1,What is the step by step guide to invest in sh...,14,1
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...,8,3
2,5,How can I increase the speed of my internet co...,14,5
3,7,Why am I mentally very lonely? How can I solve...,11,7
4,9,"Which one dissolve in water quikly sugar, salt...",13,9


In [36]:
all_ques_df.new_qid

(198429, 4)

In [51]:
search_df = all_ques_df.query('new_qid < 121424')

In [52]:
search_df = search_df.drop_duplicates('new_qid')

In [53]:
qid_1_question = search_df.query('qid == 1').questions.values[0]

In [55]:
search_df = search_df[['questions','qid']]

In [56]:
search_df.head()

,questions,qid
0,What is the step by step guide to invest in sh...,1
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,3
2,How can I increase the speed of my internet co...,5
3,Why am I mentally very lonely? How can I solve...,7
4,"Which one dissolve in water quikly sugar, salt...",9


In [57]:
search_df['qid_1_question'] = qid_1_question

In [59]:
search_df.head()

,questions,qid,qid_1_question
0,What is the step by step guide to invest in sh...,1,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,3,What is the step by step guide to invest in sh...
2,How can I increase the speed of my internet co...,5,What is the step by step guide to invest in sh...
3,Why am I mentally very lonely? How can I solve...,7,What is the step by step guide to invest in sh...
4,"Which one dissolve in water quikly sugar, salt...",9,What is the step by step guide to invest in sh...


In [60]:
search_text = search_df[['qid_1_question', 'questions']].values

In [62]:
(search_input_ids, search_input_masks, search_segment_ids) = tokenizer.predict(search_text)

In [66]:
prediction = bert_model.predict([search_input_ids, search_input_masks, search_segment_ids], verbose=1, batch_size=512)

106103/106103 [==============================] - 303s 3ms/step


In [76]:
top100_ind = prediction.argsort(axis=0)[::-1][:100,0]

In [80]:
pd.set_option('max_colwidth', 200)
search_df.iloc[top100_ind].to_csv('top100_prediction_bert.csv')